# Build a machine learning model that predict borrower's payment ability in Lending Club

In this project, we will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk. Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace [here](https://www.lendingclub.com/company/about-us).

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back.

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiving a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off before they see a return in money. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition to the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.

![img](http://cdn.biblemoneymatters.com/wp-content/uploads/2009/08/how-social-lending-works.jpg)

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this course, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

Lending Club releases data for all of the approved and declined loan applications periodically on their website. You can select a few different year ranges to download the datasets (in CSV format) for both approved and declined loans.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

- Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

Before we can start doing machine learning, we need to define what features we want to use and which column represents the target column we want to predict. Let's start by reading in the dataset and exploring it.

## Data Cleaning

In this project, we'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off.

In [1]:
#read in the data
import pandas as pd
loans_2007 = pd.read_csv("loans_2007.csv").copy()

C:\Users\parks\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
loans_2007.shape

(42538, 52)

Our data has 42538 rows and 52 columns. 

The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the [data dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097) to become familiar with what each column represents. As we understand each feature, we want to pay attention to any features that:

- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans.

Let's focus on just columns that we need to remove from consideration. Then, we can circle back and further dissect the columns we decided to keep.

Let's look at first 18 columns:
<table>
<thead>
  <tr>
    <th>name</th>
    <th>dtype</th>
    <th>first value</th>
    <th>description</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>id</td>
    <td>object</td>
    <td>1077501</td>
    <td>A unique LC assigned ID for the loan listing.</td>
  </tr>
  <tr>
    <td>member_id</td>
    <td>float64</td>
    <td>1.2966e+06</td>
    <td>A unique LC assigned Id for the borrower member.</td>
  </tr>
  <tr>
    <td>loan_amnt</td>
    <td>float64</td>
    <td>5000</td>
    <td>The listed amount of the loan applied for by the borrower.</td>
  </tr>
  <tr>
    <td>funded_amnt</td>
    <td>float64</td>
    <td>5000</td>
    <td>The total amount committed to that loan at that point in time.</td>
  </tr>
  <tr>
    <td>funded_amnt_inv</td>
    <td>float64</td>
    <td>49750</td>
    <td>The total amount committed by investors for that loan at that point in time.</td>
  </tr>
  <tr>
    <td>term</td>
    <td>object</td>
    <td>36 months</td>
    <td>The number of payments on the loan. Values are in months and can be either 36 or 60.</td>
  </tr>
  <tr>
    <td>int_rate</td>
    <td>object</td>
    <td>10.65%</td>
    <td>Interest Rate on the loan</td>
  </tr>
  <tr>
    <td>installment</td>
    <td>float64</td>
    <td>162.87</td>
    <td>The monthly payment owed by the borrower if the loan originates.</td>
  </tr>
  <tr>
    <td>grade</td>
    <td>object</td>
    <td>B</td>
    <td>LC assigned loan grade</td>
  </tr>
  <tr>
    <td>sub_grade</td>
    <td>object</td>
    <td>B2</td>
    <td>LC assigned loan subgrade</td>
  </tr>
  <tr>
    <td>emp_title</td>
    <td>object</td>
    <td>NaN</td>
    <td>The job title supplied by the Borrower when applying for the loan.</td>
  </tr>
  <tr>
    <td>emp_length</td>
    <td>object</td>
    <td>10+ years</td>
    <td>Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.</td>
  </tr>
  <tr>
    <td>home_ownership</td>
    <td>object</td>
    <td>RENT</td>
    <td>The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.</td>
  </tr>
  <tr>
    <td>annual_inc</td>
    <td>float64</td>
    <td>24000</td>
    <td>The self-reported annual income provided by the borrower during registration.</td>
  </tr>
  <tr>
    <td>verification_status</td>
    <td>object</td>
    <td>Verified</td>
    <td>Indicates if income was verified by LC, not verified, or if the income source was verified</td>
  </tr>
  <tr>
    <td>issue_d</td>
    <td>object</td>
    <td>Dec-2011</td>
    <td>The month which the loan was funded</td>
  </tr>
  <tr>
    <td>loan_status</td>
    <td>object</td>
    <td>Charged Off</td>
    <td>Current status of the loan</td>
  </tr>
  <tr>
    <td>pymnt_plan</td>
    <td>object</td>
    <td>n</td>
    <td>Indicates if a payment plan has been put in place for the loan</td>
  </tr>
  <tr>
    <td>purpose</td>
    <td>object</td>
    <td>car</td>
    <td>A category provided by the borrower for the loan request.</td>
  </tr>
</tbody>
</table>

After analyzing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completely funded)

Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the **grade** and **sub_grade** values are categorical, the **int_rate** column contains continuous values, which are better suited for machine learning.

In [4]:
#Drop the columns listed above
drop_columns=["id","member_id","funded_amnt","funded_amnt_inv",
              "grade","sub_grade","emp_title","issue_d"]
loans_2007=loans_2007.drop(drop_columns, axis=1)
loans_2007.shape[1]

44

Let's now look at the next 18 columns:
<table>
<thead>
  <tr>
    <th>name</th>
    <th>dtype</th>
    <th>first value</th>
    <th>description</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>title</td>
    <td>object</td>
    <td>Computer</td>
    <td>The loan title provided by the borrower</td>
  </tr>
  <tr>
    <td>zip_code</td>
    <td>object</td>
    <td>860xx</td>
    <td>The first 3 numbers of the zip code provided by the borrower in the loan application.</td>
  </tr>
  <tr>
    <td>addr_state</td>
    <td>object</td>
    <td>AZ</td>
    <td>The state provided by the borrower in the loan application</td>
  </tr>
  <tr>
    <td>dti</td>
    <td>float64</td>
    <td>27.65</td>
    <td>A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.</td>
  </tr>
  <tr>
    <td>delinq_2yrs</td>
    <td>float64</td>
    <td>0</td>
    <td>The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years</td>
  </tr>
  <tr>
    <td>earliest_cr_line</td>
    <td>object</td>
    <td>Jan-1985</td>
    <td>The month the borrower's earliest reported credit line was opened</td>
  </tr>
  <tr>
    <td>inq_last_6mths</td>
    <td>float64</td>
    <td>1</td>
    <td>The number of inquiries in past 6 months (excluding auto and mortgage inquiries)</td>
  </tr>
  <tr>
    <td>open_acc</td>
    <td>float64</td>
    <td>3</td>
    <td>The number of open credit lines in the borrower's credit file.</td>
  </tr>
  <tr>
    <td>pub_rec</td>
    <td>float64</td>
    <td>0</td>
    <td>Number of derogatory public records</td>
  </tr>
  <tr>
    <td>revol_bal</td>
    <td>float64</td>
    <td>13648</td>
    <td>Total credit revolving balance</td>
  </tr>
  <tr>
    <td>revol_util</td>
    <td>object</td>
    <td>83.7%</td>
    <td>Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.</td>
  </tr>
  <tr>
    <td>total_acc</td>
    <td>float64</td>
    <td>9</td>
    <td>The total number of credit lines currently in the borrower's credit file</td>
  </tr>
  <tr>
    <td>initial_list_status</td>
    <td>object</td>
    <td>f</td>
    <td>The initial listing status of the loan. Possible values are – W, F</td>
  </tr>
  <tr>
    <td>out_prncp</td>
    <td>float64</td>
    <td>0</td>
    <td>Remaining outstanding principal for total amount funded</td>
  </tr>
  <tr>
    <td>out_prncp_inv</td>
    <td>float64</td>
    <td>0</td>
    <td>Remaining outstanding principal for portion of total amount funded by investors</td>
  </tr>
  <tr>
    <td>total_pymnt</td>
    <td>float64</td>
    <td>5863.16</td>
    <td>Payments received to date for total amount funded</td>
  </tr>
  <tr>
    <td>total_pymnt_inv</td>
    <td>float64</td>
    <td>5833.84</td>
    <td>Payments received to date for portion of total amount funded by investors</td>
  </tr>
  <tr>
    <td>total_rec_prncp</td>
    <td>float64</td>
    <td>5000</td>
    <td>Principal received to date</td>
  </tr>
</tbody>
</table>

Within this group of columns, we need to drop the following columns:

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

In [5]:
drop_colomn=["zip_code","out_prncp","out_prncp_inv",
             "total_pymnt","total_pymnt_inv","total_rec_prncp"]
loans_2007=loans_2007.drop(drop_colomn,axis=1)

Let's now move on to the last group of features:
<table>
<thead>
  <tr>
    <th>name</th>
    <th>dtype</th>
    <th>first value</th>
    <th>description</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>total_rec_int</td>
    <td>float64</td>
    <td>863.16</td>
    <td>Interest received to date</td>
  </tr>
  <tr>
    <td>total_rec_late_fee</td>
    <td>float64</td>
    <td>0</td>
    <td>Late fees received to date</td>
  </tr>
  <tr>
    <td>recoveries</td>
    <td>float64</td>
    <td>0</td>
    <td>post charge off gross recovery</td>
  </tr>
  <tr>
    <td>collection_recovery_fee</td>
    <td>float64</td>
    <td>0</td>
    <td>post charge off collection fee</td>
  </tr>
  <tr>
    <td>last_pymnt_d</td>
    <td>object</td>
    <td>Jan-2015</td>
    <td>Last month payment was received</td>
  </tr>
  <tr>
    <td>last_pymnt_amnt</td>
    <td>float64</td>
    <td>171.62</td>
    <td>Last total payment amount received</td>
  </tr>
  <tr>
    <td>last_credit_pull_d</td>
    <td>object</td>
    <td>Jun-2016</td>
    <td>The most recent month LC pulled credit for this loan</td>
  </tr>
  <tr>
    <td>collections_12_mths_ex_med</td>
    <td>float64</td>
    <td>0</td>
    <td>Number of collections in 12 months excluding medical collections</td>
  </tr>
  <tr>
    <td>policy_code</td>
    <td>float64</td>
    <td>1</td>
    <td>publicly available policy_code=1 new products not publicly available policy_code=2</td>
  </tr>
  <tr>
    <td>application_type</td>
    <td>object</td>
    <td>INDIVIDUAL</td>
    <td>Indicates whether the loan is an individual application or a joint application with two co-borrowers</td>
  </tr>
  <tr>
    <td>acc_now_delinq</td>
    <td>float64</td>
    <td>0</td>
    <td>The number of accounts on which the borrower is now delinquent.</td>
  </tr>
  <tr>
    <td>chargeoff_within_12_mths</td>
    <td>float64</td>
    <td>0</td>
    <td>Number of charge-offs within 12 months</td>
  </tr>
  <tr>
    <td>delinq_amnt</td>
    <td>float64</td>
    <td>0</td>
    <td>The past-due amount owed for the accounts on which the borrower is now delinquent.</td>
  </tr>
  <tr>
    <td>pub_rec_bankruptcies</td>
    <td>float64</td>
    <td>0</td>
    <td>Number of public record bankruptcies</td>
  </tr>
  <tr>
    <td>tax_liens</td>
    <td>float64</td>
    <td>0</td>
    <td>Number of tax liens</td>
  </tr>
  <tr>
    <td>issue_d</td>
    <td>object</td>
    <td>Dec-2011</td>
    <td>The month which the loan was funded</td>
  </tr>
  <tr>
    <td>loan_status</td>
    <td>object</td>
    <td>Charged Off</td>
    <td>Current status of the loan</td>
  </tr>
  <tr>
    <td>pymnt_plan</td>
    <td>object</td>
    <td>n</td>
    <td>Indicates if a payment plan has been put in place for the loan</td>
  </tr>
  <tr>
    <td>purpose</td>
    <td>object</td>
    <td>car</td>
    <td>A category provided by the borrower for the loan request.</td>
  </tr>
</tbody>
</table>

In the last group of columns, we need to drop the following columns:

- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [6]:
drop_columns=['total_rec_int','total_rec_late_fee','recoveries',
              'collection_recovery_fee','last_pymnt_d','last_pymnt_amnt']
loans_2007=loans_2007.drop(drop_columns,axis=1)

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

We should use the **loan_status** column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [7]:
loans_2007["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

There are 8 different possible values for the loan_status column. You can read about most of the different loan statuses on the Lending Clube webste. The two values that start with "Does not meet the credit policy" aren't explained unfortunately. A quick Google search takes us to explanations from the lending community [here](https://forum.lendacademy.com/?topic=2427.msg20813#msg20813).

<table>
<thead>
  <tr>
    <th>Loan Status</th>
    <th>dtype</th>
    <th>first value</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Fully Paid</td>
    <td>33136</td>
    <td>Loan has been fully paid off.</td>
  </tr>
  <tr>
    <td>Charged Off</td>
    <td>5634</td>
    <td>Loan for which there is no longer a reasonable expectation of further payments.</td>
  </tr>
  <tr>
    <td>Does not meet the credit policy. Status:Fully Paid</td>
    <td>1988</td>
    <td>While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.</td>
  </tr>
  <tr>
    <td>Does not meet the credit policy. Status:Charged Off</td>
    <td>761</td>
    <td>While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.</td>
  </tr>
  <tr>
    <td>In Grace Period</td>
    <td>20</td>
    <td>The loan is past due but still in the grace period of 15 days.</td>
  </tr>
  <tr>
    <td>Late (16-30 days)</td>
    <td>8</td>
    <td>Loan hasn't been paid in 16 to 30 days (late on the current payment).</td>
  </tr>
  <tr>
    <td>Late (31-120 days)</td>
    <td>24</td>
    <td>Loan hasn't been paid in 31 to 120 days (late on the current payment).</td>
  </tr>
  <tr>
    <td>Current</td>
    <td>961</td>
    <td>Loan is up to date on current payments.</td>
  </tr>
  <tr>
    <td>Default</td>
    <td>3</td>
    <td>Loan is defaulted on and no payment has been made for more than 121 days.</td>
  </tr>
</tbody>
</table>

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still ongoing and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a *binary classification* one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case.

In [8]:
#Keep only the rows with loan status Fully Paid and Charged Off
loans_2007=loans_2007[(loans_2007["loan_status"] == "Fully Paid") | (loans_2007["loan_status"] == "Charged Off")]

In [9]:
#replace the string values with 0 and 1
status_replace = {
    "loan_status" : {
        "Fully Paid" : 1,
        "Charged Off" : 0,
    }
}

loans_2007=loans_2007.replace(status_replace)

In [10]:
#Drop all columns that with only one unique value beside NaN value
drop_columns = []

for column in loans_2007.columns:
    non_null = loans_2007[column].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique == 1:
        drop_columns.append(column)

filtered_loans_2007=loans_2007.drop(drop_columns, axis=1).copy()
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


It looks we were able to remove 9 more columns since they only contained 1 unique value.

## Preparing the features

We removed all of the columns that contained redundant information, weren't useful for modeling, required too much processing to make useful, or leaked information from the future. Now, we'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process.

In [11]:
#change to easier name
loans = filtered_loans_2007
#check out the columns with null values
null_counts = loans.isnull().sum()
print(null_counts[null_counts > 0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


While most of the columns have no missing values, two columns have fifty or less rows with missing values, and two columns, **emp_length** and **pub_rec_bankruptcies**, contain a relatively high amount of missing values.

Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

In [12]:
#inspect the values of the column pub_rec_bankruptcies
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We see that this column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. Let's drop it. In addition, we'll remove the remaining rows containing null values.

In [13]:
#drop the whole column for pub_rec_bankruptcies
loans = loans.drop("pub_rec_bankruptcies", axis=1)
#drop only the rows with missing values
loans = loans.dropna(axis=0)
#check counts for each column data type
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth.

In [14]:
object_columns_df = loans.select_dtypes(include=["object"])

In [15]:
print(object_columns_df.head())

         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   
1   60 months   15.27%   < 1 year           RENT     Source Verified   
2   36 months   15.96%  10+ years           RENT        Not Verified   
3   36 months   13.49%  10+ years           RENT     Source Verified   
5   36 months    7.90%    3 years           RENT     Source Verified   

          purpose                                  title addr_state  \
0     credit_card                               Computer         AZ   
1             car                                   bike         GA   
2  small_business                   real estate business         IL   
3           other                               personel         CA   
5         wedding  My wedding loan I promise to pay back         AZ   

  earliest_cr_line revol_util last_credit_pull_d  
0         Jan-1985      83.7%           Jun-2016  
1         Apr-1999       9.4%         

Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

- home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
- verification_status: indicates if income was verified by Lending Club,
- emp_length: number of years the borrower was employed upon time of application,
- term: number of payments on the loan, either 36 or 60,
- addr_state: borrower's state of residence,
- purpose: a category provided by the borrower for the loan request,
- title: loan title provided by the borrower,

There are also some columns that represent numeric values, that need to be converted:

- int_rate: interest rate of the loan in %,
- revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more here.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

- earliest_cr_line: The month the borrower's earliest reported credit line was opened,
- last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe

In [16]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for col in cols:
    print(loans[col].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  

The **home_ownership**, **verification_status**, **emp_length**, **term**, and **addr_state columns** all contain multiple discrete values. We should clean the **emp_length column** and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the **purpose** and **title** columns to understand which column we want to keep.

In [17]:
for col in ["title", "purpose"]:
    print(loans[col].value_counts())

Debt Consolidation           2068
Debt Consolidation Loan      1599
Personal Loan                 624
Consolidation                 488
debt consolidation            466
                             ... 
DEBT PAYOFFS                    1
My First LendingClub Loan       1
Clear my credit card debt       1
HOME IMPROVEMENT PROJECT        1
new kitchen                     1
Name: title, Length: 18881, dtype: int64
debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64


The **home_ownership**, **verification_status**, **emp_length**, and **term** columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the **purpose** and **title** columns do contain overlapping information but we'll keep the **purpose** column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

In [18]:
#Remove the last_credit_pull_d, addr_state, title, and earliest_cr_line columns from loans
cols=['last_credit_pull_d', 'addr_state', 'title','earliest_cr_line']
loans = loans.drop(cols, axis=1)
#remove % from int_rate, revol_util column and change the dtype to float
loans["int_rate"] = loans['int_rate'].str.rstrip("%").astype('float')
loans["revol_util"] = loans['revol_util'].str.rstrip("%").astype('float')

In [19]:
#Clean emp_length columns
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.replace(mapping_dict)

Let's now encode the **home_ownership**, **verification_status**, **purpose**, and **term** columns as dummy variables so we can use them in our model. We first need to use the Pandas get_dummies method to return a new Dataframe containing a new column for each dummy variable:

In [20]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

## Make Predictions

As we prepared the data, we removed columns that had data leakage issues, contained redundant information, or required additional processing to turn into useful features. We cleaned features that had formatting issues, and converted categorical columns to dummy variables.

We noticed that there's a class imbalance in our target column, loan_status. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. 

In [21]:
loans = pd.read_csv("cleaned_loans_2007.csv")
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37675 entries, 0 to 37674
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37675 non-null  float64
 1   int_rate                             37675 non-null  float64
 2   installment                          37675 non-null  float64
 3   emp_length                           37675 non-null  int64  
 4   annual_inc                           37675 non-null  float64
 5   loan_status                          37675 non-null  int64  
 6   dti                                  37675 non-null  float64
 7   delinq_2yrs                          37675 non-null  float64
 8   inq_last_6mths                       37675 non-null  float64
 9   open_acc                             37675 non-null  float64
 10  pub_rec                              37675 non-null  float64
 11  revol_bal                   

Before we dive into machine learning model, we need to think about how we deal with the error.
<table>
<thead>
  <tr>
    <th>Loan Actual Status</th>
    <th>Prediction</th>
    <th>Error Type</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>0</td>
    <td>1</td>
    <td>False Positive</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1</td>
    <td>True Positive</td>
  </tr>
  <tr>
    <td>0</td>
    <td>0</td>
    <td>True Negative</td>
  </tr>
  <tr>
    <td>1</td>
    <td>0</td>
    <td>False Negative</td>
  </tr>
</tbody>
</table>

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off. Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

From above, it seems that we should optimize for:
- high recall (true positive rate)
- low fall-out (false positive rate)

Since there are huge imbalance in the data, let's start with prediction that says all 1s.

In [22]:
import numpy as np
predictions = pd.Series(np.ones(loans.shape[0]))

False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off:

- fpr = fp / (fp + tn)

True positive rate is the number of true positives divided by the number of true positives plus the number of false negatives. This divides all the cases where we thought a loan would be paid off and it was by all the loans that were paid off:

- tpr = tp / (tp + fn)

Simple english ways to think of each term are:

False Positive Rate: "the percentage of the loans that shouldn't be funded that I would fund".

True Positive Rate: "the percentage of loans that should be funded that I would fund".

Generally, if we want to reduce false positive rate, true positive rate will also go down. This is because if we want to reduce the risk of false positives, we wouldn't think about funding riskier loans in the first place.

In [23]:
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

fpr = fp/(fp+tn)
tpr = tp/(tp+fn)

In [24]:
print(fpr)
print(tpr)

1.0
1.0


This is because we predicted 1 for each row. This means that we correctly identified all of the good loans (true positive rate), but we also incorrectly identified all of the bad loans (false positive rate). Now that we've setup error metrics, we can move on to making predictions using a machine learning algorithm.

First, let's try to make machine learing model with simple Logistic regression model

In [25]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=200)
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)

While we generated predictions above, those predictions were overfit. They were overfit because we generated predictions using the same data that we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers.
In order to get a realistic depiction of the accuracy of the model, let's perform k-fold cross validation.

In [26]:
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(max_iter=200)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

C:\Users\parks\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9986371802019451
0.9959176099461867


As you can see from the above, our fpr and tpr are around what we'd expect if the model was predicting all ones.
Unfortunately, even though we're not using accuracy as an error metric, the classifier is, and it isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes. The two main ways are:

- Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
- Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

We will use second method. We can do this by setting the class_weight parameter to balanced when creating the LogisticRegression instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(max_iter=200,class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.5418137892585021
0.31712748190758955


While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything.

In [28]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(max_iter=200,class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.18329926283838197
0.07849322694377435


It looks like assigning manual penalties lowered the false positive rate to 8%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

While we could tweak the penalties further, it's best to move to trying a different model right now, for larger potential false positive rate gains. We can always loop back and iterate on the penalties more later.

Let's try a more complex algorithm, random forest. We learned about random forests in a previous mission, and constructed our own model. Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.

In [31]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9974601994672614
0.989979588049731


## Conclusion

Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of nearly 8%, and a true positive rate of nearly 18%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 8% of borrowers defaulting, and that the pool of 18% of borrowers is large enough to make enough interest money to offset the losses.

Although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.